# WP2.1 - Short circuit on three-phase passive grids

This Notebook shows exemplary tests that are implemented in the sce_test/test_sce.py file.

See: https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

Therefore it must be run in an environment where the pandapower test folders can be found by the jupyter server's python environment.
This can be done by checking out the git branch sce in pandapower and locally initializing the repository by using `pip install -e .`.

Further information on running short-circuit calculations in pandapower can be found here:

https://pandapower.readthedocs.io/en/latest/shortcircuit/run.html

https://github.com/e2nIEE/pandapower/blob/sce/tutorials/shortcircuit/shortcircuit.ipynb

In [18]:
import os
from itertools import product

from pandapower import pp_dir
from pandapower.test.shortcircuit.sce_tests.functions_tests import load_test_case_data, run_test_cases, compare_results

testfiles_path = os.path.join(pp_dir, 'test', 'shortcircuit', 'sce_tests')

In [19]:
import warnings
# Ignore all future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all runtime warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

From the defined parameters for faults, cases, network names and vector_groups as well as the values for r_faults and x_faults all combinations are generated to run as a test.

In [20]:
# Define common parameters
net_names = [
    "test_case_2_five_bus_radial_grid",
    "test_case_3_five_bus_meshed_grid",
    "test_case_4_twenty_bus_radial_grid"
]
faults = ["LLL", "LL", "LLG", "LG"]
cases = ["max", "min"]
values = [(0.0, 0.0), (5.0, 5.0)]
vector_groups = ['Dyn', 'Yyn', 'YNyn']
lv_tol_percents = [6, 10]
fault_location_buses = [0, 1, 2, 3]
is_branch_test = [False, True]

# Create parameter list
parametrize_values = list(product(faults, cases, values, lv_tol_percents, fault_location_buses, is_branch_test))

# Create parameter list with vector group
parametrize_values_vector = list(product(
    net_names, faults[:2], cases, values, lv_tol_percents, vector_groups[:1], fault_location_buses, is_branch_test
))+list(product(
    net_names, faults[2:], cases, values, lv_tol_percents, vector_groups, fault_location_buses, is_branch_test
))


total_test_count = len(parametrize_values)+len(parametrize_values_vector)
formatted_output = (
    f"Number of parameterization values: {len(parametrize_values)}\n"
    f"Number of parameterization value vectors: {len(parametrize_values_vector)}\n"
    f"Total number of tests: {total_test_count}"
)
print(formatted_output)

Number of parameterization values: 256
Number of parameterization value vectors: 1536
Total number of tests: 1792


The `parametrize_values` list and the `parametrize_values_vector` contain combinations for the tests.
Tests for "LL" and "LLL" using different vector groups are omitted as these have no influence on the results.
There is one test using `parametrize_values` and three using `parametrize_values_vector`.
Shown above are the amount of test for each of the lists as well as the total.

In this Notebook we will only show a exemplary examples for each test case and vector group.
The test will load the expected results from the corresponding excel file (exported from commercial PowerFactory software results).
Then the calculations are run in pandapower using the parameters from the list of permutation and compared against the expected results.
Running the tests in the file below, it is possible to execute the complete list of tests.

https://github.com/e2nIEE/pandapower/blob/sce/pandapower/test/shortcircuit/SCE_Tests/test_all_faults_and_cases.py

---

## Test Case 1: Four bus radial grid

![four_bus_radial](four_bus_radial.png)

In [21]:
net_name = "test_case_1_four_bus_radial_grid"

fault = "LLL"
case = "max"
fault_values = (0.0, 0.0)
# lv_tol_percents does not influence max case (default value is 10)
lv_tol_percents = 10
fault_location_bus = 0
is_branch_test = False
# vector group not necesarry as the grid does not contain a transformer

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus)

These are all the test results that were exported from PowerFactory. Depending on the parameter combination, the corresponding dataframe is selected.

In [22]:
dataframes["bus"]["LLL_max_10"]

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163
1,Bus_1,0.000000,0.000000,0.000000,0.000000
2,Bus_2,0.000000,0.000000,0.000000,0.000000
3,Bus_3,0.000000,0.000000,0.000000,0.000000


Let's now select the dataframe that we need in order to execute our first test case.

With the loaded network and expected results we now can run the test to receive the dataframes and the columns that should be compared.

Below is the list of columns that will be compared.

In [23]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['name', 'ikss_ka', 'skss_mw', 'rk_ohm', 'xk_ohm'], dtype='object')

In [24]:
net_df[columns_to_check]

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886751,100.0,0.437816,4.378164


In [25]:
# dataframes["bus"]["LLL_max_10"]
pf_results[columns_to_check]

,name,ikss_ka,skss_mw,rk_ohm,xk_ohm
0,Bus_0,2.886752,100.000006,0.437816,4.378163


As you can see the results are matching and the comparison function will pass and not throw any errors.

In [26]:
compare_results(columns_to_check, net_df, pf_results)

## Test Case 2: Five bus radial grid

![five_bus_radial](five_bus_radial.png)

In [27]:
net_name = "test_case_2_five_bus_radial_grid"

fault = "LG"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 3
is_branch_test = True
vector_group = 'Dyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [28]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_a_from_ka', 'ikss_a_from_degree', 'ikss_a_to_ka',
       'ikss_a_to_degree', 'p_a_from_mw', 'q_a_from_mvar', 'p_a_to_mw',
       'q_a_to_mvar', 'vm_a_from_pu', 'va_a_from_degree', 'vm_a_to_pu',
       'va_a_to_degree', 'ikss_b_from_ka', 'ikss_b_from_degree',
       'ikss_b_to_ka', 'ikss_b_to_degree', 'p_b_from_mw', 'q_b_from_mvar',
       'p_b_to_mw', 'q_b_to_mvar', 'vm_b_from_pu', 'va_b_from_degree',
       'vm_b_to_pu', 'va_b_to_degree', 'ikss_c_from_ka', 'ikss_c_from_degree',
       'ikss_c_to_ka', 'ikss_c_to_degree', 'p_c_from_mw', 'q_c_from_mvar',
       'p_c_to_mw', 'q_c_to_mvar', 'vm_c_from_pu', 'va_c_from_degree',
       'vm_c_to_pu', 'va_c_to_degree'],
      dtype='object')

In [29]:
net_df[columns_to_check]

,ikss_a_from_ka,ikss_a_from_degree,ikss_a_to_ka,ikss_a_to_degree,p_a_from_mw,q_a_from_mvar,p_a_to_mw,q_a_to_mvar,vm_a_from_pu,va_a_from_degree,...,ikss_c_to_ka,ikss_c_to_degree,p_c_from_mw,q_c_from_mvar,p_c_to_mw,q_c_to_mvar,vm_c_from_pu,va_c_from_degree,vm_c_to_pu,va_c_to_degree
0,0.823769,-37.522885,0.823628,142.506266,7.494644,4.592405,-5.436024,-3.997649,0.924064,-6.024653,...,0.002161,66.826551,-0.019621,-0.02339,0.016470,0.023266,1.016206,117.549409,1.142526,121.532466
1,0.823097,142.652000,0.823405,-37.435411,-3.387439,-3.387439,5.438615,3.991031,0.504042,7.652000,...,0.001293,-113.594362,0.000000,0.00000,-0.009755,-0.013998,1.272497,124.745443,1.142526,121.532466
2,0.000000,0.000000,0.000868,-112.545962,0.000000,0.000000,-0.002591,0.006618,0.709551,-1.141325,...,0.000868,-112.545962,0.000000,0.00000,-0.006715,-0.009268,1.142727,121.523260,1.142526,121.532466


In [30]:
# dataframes["branch"]["LG_min_fault_6"]
pf_results[columns_to_check]

,ikss_a_from_ka,ikss_a_from_degree,ikss_a_to_ka,ikss_a_to_degree,p_a_from_mw,q_a_from_mvar,p_a_to_mw,q_a_to_mvar,vm_a_from_pu,va_a_from_degree,...,ikss_c_to_ka,ikss_c_to_degree,p_c_from_mw,q_c_from_mvar,p_c_to_mw,q_c_to_mvar,vm_c_from_pu,va_c_from_degree,vm_c_to_pu,va_c_to_degree
0,0.823769,-37.522885,0.823628,142.506266,7.494644e+00,4.592405e+00,-5.436024,-3.997649,0.924064,-6.024653,...,0.002161,66.826546,-1.962060e-02,-2.338974e-02,0.016470,0.023266,1.016206,117.549409,1.142526,121.532466
1,0.823097,142.652000,0.823405,-37.435411,-3.387439e+00,-3.387439e+00,5.438615,3.991031,0.504042,7.652001,...,0.001293,-113.594366,-1.287415e-09,3.859960e-10,-0.009755,-0.013998,1.272497,124.745443,1.142526,121.532466
2,0.000000,0.000000,0.000868,-112.545967,7.551381e-10,-5.182125e-11,-0.002591,0.006618,0.709551,-1.141325,...,0.000868,-112.545964,-5.995053e-10,1.113843e-10,-0.006715,-0.009268,1.142727,121.523260,1.142526,121.532466


In [31]:
compare_results(columns_to_check, net_df, pf_results)

## Test Case 3: Five bus meshed grid

![five_bus_meshed](five_bus_meshed.png)

In [32]:
net_name = "test_case_3_five_bus_meshed_grid"

fault = "LLL"
case = "min"
fault_values = (5.0, 5.0)
lv_tol_percents = 6
fault_location_bus = 2
is_branch_test = True
vector_group = 'Yyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [33]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['name', 'ikss_ka', 'ikss_a_from_ka', 'ikss_a_from_degree',
       'ikss_a_to_ka', 'ikss_a_to_degree', 'p_a_from_mw', 'q_a_from_mvar',
       'p_a_to_mw', 'q_a_to_mvar', 'vm_a_from_pu', 'va_a_from_degree',
       'vm_a_to_pu', 'va_a_to_degree'],
      dtype='object')

In [34]:
net_df[columns_to_check]

,name,ikss_ka,ikss_a_from_ka,ikss_a_from_degree,ikss_a_to_ka,ikss_a_to_degree,p_a_from_mw,q_a_from_mvar,p_a_to_mw,q_a_to_mvar,vm_a_from_pu,va_a_from_degree,vm_a_to_pu,va_a_to_degree
0,Line_0,1.146247,1.146247,-52.634345,1.146247,127.365655,23.643589,22.033819,-19.708246,-19.708246,0.81393,-9.652736,0.70193,-7.634345
1,Line_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.70193,-7.634345,0.70193,-7.634345
2,Line_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.70193,-7.634345,0.70193,-7.634345
3,Line_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.70193,-7.634345,0.70193,-7.634345


In [35]:
# dataframes["branch"]["LLL_min_fault_6"]
pf_results[columns_to_check]

,name,ikss_ka,ikss_a_from_ka,ikss_a_from_degree,ikss_a_to_ka,ikss_a_to_degree,p_a_from_mw,q_a_from_mvar,p_a_to_mw,q_a_to_mvar,vm_a_from_pu,va_a_from_degree,vm_a_to_pu,va_a_to_degree
0,Line_0,1.146247,1.146247,-52.634345,1.146247,127.365655,23.643589,22.033819,-1.970825e+01,-1.970825e+01,0.81393,-9.652736,0.70193,-7.634345
1,Line_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.233007e-09,1.025774e-09,0.70193,-7.634345,0.70193,-7.634345
2,Line_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.233008e-09,1.025769e-09,0.70193,-7.634345,0.70193,-7.634345
3,Line_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.70193,-7.634345,0.70193,-7.634345


In [36]:
compare_results(columns_to_check, net_df, pf_results)

## Test Case 4: Twenty bus radial grid

![twenty_bus_grid](twenty_bus_grid.png)

In [37]:
net_name = "test_case_4_twenty_bus_radial_grid"

fault = "LG"
case = "max"
fault_values = (0.0, 0.0)
lv_tol_percents = 10
fault_location_bus = 3
is_branch_test = False
vector_group = 'YNyn'

# loading the network and testing data
net, dataframes = load_test_case_data(net_name, fault_location_bus, vector_group)

In [38]:
columns_to_check, net_df, pf_results = run_test_cases(net, dataframes["branch" if is_branch_test else "bus"], fault, case, fault_values, lv_tol_percents, fault_location_bus, is_branch_test)
columns_to_check

Index(['ikss_ka', 'skss_mw', 'rk0_ohm', 'xk0_ohm', 'rk1_ohm', 'xk1_ohm',
       'rk2_ohm', 'xk2_ohm'],
      dtype='object')

In [39]:
net_df[columns_to_check]

,ikss_ka,skss_mw,rk0_ohm,xk0_ohm,rk1_ohm,xk1_ohm,rk2_ohm,xk2_ohm
3,6.628723,1.530838,0.00899,0.021509,0.012035,0.044302,0.012035,0.044302


In [40]:
# dataframes["bus"]["LG_max_10"]
pf_results[columns_to_check]

,ikss_ka,skss_mw,rk0_ohm,xk0_ohm,rk1_ohm,xk1_ohm,rk2_ohm,xk2_ohm
13,6.628723,1.530838,0.00899,0.021509,0.012035,0.044302,0.012035,0.044302


In [41]:
compare_results(columns_to_check, net_df, pf_results)